In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from transformers import MarianMTModel, MarianTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import re

In [3]:
df = pd.read_csv("/content/CleanedReviews.csv")
df.shape

(43088, 2)

In [4]:
df.isna().sum()

Rating     0
Review    34
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.isna().sum()

Rating    0
Review    0
dtype: int64

In [7]:
df.Review[1]

'unable apply discount code purchase process assistance swift courteous eager continue shopping website excellent service'

In [8]:
mname = "marefa-nlp/marefa-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62802, 512, padding_idx=62801)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62802, 512, padding_idx=62801)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [11]:
from tqdm import tqdm
def translate_texts(texts, tokenizer, model, device):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    translated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return translated_texts

# Load your data

# Parameters for batch processing
batch_size = 32  # Adjust based on your GPU memory
translated_texts = []
total_translated_count = 0

for start_idx in tqdm(range(0, len(df), batch_size)):
    end_idx = min(start_idx + batch_size, len(df))
    batch_texts = df['Review'][start_idx:end_idx].tolist()

    translated_batch = translate_texts(batch_texts, tokenizer, model, device)
    translated_texts.extend(translated_batch)

    total_translated_count += len(translated_batch)

    # Print progress every 500 translations
    if total_translated_count % 500 == 0:
        print(f"{total_translated_count} reviews translated")

    # Save checkpoint every 2000 translations
    if total_translated_count % 2000 == 0:
        checkpoint_df = pd.DataFrame({
            'Rating': df['Rating'][:total_translated_count],
            'Review': df['Review'][:total_translated_count],
            'Translated_Review': translated_texts
        })
        checkpoint_df.to_csv(f'translated_reviews_checkpoint_{total_translated_count}.csv', index=False)
        print(f"Checkpoint saved at {total_translated_count} translations")

# Save the final translations to CSV
final_translated_df = pd.DataFrame({
    'Rating': df['Rating'],
    'Review': df['Review'],
    'Translated_Review': translated_texts
})
final_translated_df.to_csv('Translated_Reviews.csv', index=False)

print(final_translated_df)

  9%|▉         | 125/1346 [17:48<3:11:19,  9.40s/it]

4000 reviews translated
Checkpoint saved at 4000 translations


 19%|█▊        | 250/1346 [32:43<42:22,  2.32s/it]

8000 reviews translated
Checkpoint saved at 8000 translations


 28%|██▊       | 375/1346 [41:51<1:07:06,  4.15s/it]

...

40000 reviews translated


 93%|█████████▎| 1250/1346 [2:53:56<28:13, 17.64s/it]

Checkpoint saved at 40000 translations


100%|██████████| 1346/1346 [3:11:28<00:00,  8.54s/it]


         Rating                                             Review  \
0      Positive    perfect shipping fast really good quality usual   
1      Positive  unable apply discount code purchase process as...   
2      Positive  fantastic always problem order gym shark recti...   
3      Positive  quick delivery high quality website easy use p...   
4      Positive  quality gym clothing easy purchase delivered q...   
...         ...                                                ...   
43083  Positive                         great stuff literally best   
43084  Positive  add cart quick shipping workout clothes amazin...   
43085  Positive                                    clothes amazing   
43086  Positive  love she i nnnn pants though sized like anyway...   
43087  Positive                                           good job   

                                       Translated_Review  
0           الشحن المثالي السريع جيد الجودة المعتاد حقاً  
1      مساعدة عملية شراء شفرة الخصم غير ق